<a href="https://colab.research.google.com/github/bthu207/group-project/blob/main/keywords_searching.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. Mount Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


2. Import pakages

In [2]:
# === [1] Import packages ===
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import json

In [3]:
# === [2] Read real CSV files ===
csv_paths = [
    '/content/drive/MyDrive/[DS-B3] group project/processed_folktales.csv',
    '/content/drive/MyDrive/[DS-B3] group project/processed_articles_sep2025.csv',
    '/content/drive/MyDrive/[DS-B3] group project/processed_articles_aug_oct_2025.csv'
    ]

df_list = []
for path in csv_paths:
    try:
        temp_df = pd.read_csv(path)
        print(f" Loaded file: {path}")
        print(f"   Cột: {list(temp_df.columns)}")
        df_list.append(temp_df)
    except Exception as e:
        print(f" Lỗi khi đọc {path}: {e}")

# Gộp tất cả thành 1 DataFrame duy nhất
df = pd.concat(df_list, ignore_index=True)
print("\nTotal number of articles after merging:", len(df))
print("Merged column list:", list(df.columns))


 Loaded file: /content/drive/MyDrive/[DS-B3] group project/processed_folktales.csv
   Cột: ['title', 'content', 'url', 'section', 'source']
 Loaded file: /content/drive/MyDrive/[DS-B3] group project/processed_articles_sep2025.csv
   Cột: ['url', 'title', 'content', 'published_time', 'source', 'title_clean', 'content_clean', 'label']
 Loaded file: /content/drive/MyDrive/[DS-B3] group project/processed_articles_aug_oct_2025.csv
   Cột: ['title', 'content', 'url', 'section', 'source', 'published_time']

Total number of articles after merging: 2791
Merged column list: ['title', 'content', 'url', 'section', 'source', 'published_time', 'title_clean', 'content_clean', 'label']


In [4]:
import pandas as pd

def load_articles(csv_path):

    df = pd.read_csv(csv_path)

    # Create dict mapping: column names that may be encountered -> standard columns
    name_map = {
        "url": ["url", "link", "article_url"],
        "title": ["title", "headline", "title_clean"],
        "content": ["content", "body", "article_content", "content_clean"],
        "published_time": ["published_time", "date", "datetime", "created_at"],
        "category": ["category", "section", "topic"],
        "label": ["label", "target", "age_group", "safe_label"],
        "section": ["section"]
    }

    # find columns having similar names
    def find_column(possibles, columns):
        for p in possibles:
            for c in columns:
                if p.lower() == c.lower():
                    return c
        return None

    selected = {}
    for std_col, poss in name_map.items():
        real_col = find_column(poss, df.columns)
        if real_col:
            selected[std_col] = real_col

    df_clean = pd.DataFrame()
    for std_col, real_col in selected.items():
        df_clean[std_col] = df[real_col]

    # Thêm cột còn thiếu (giá trị rỗng)
    for col in name_map.keys():
        if col not in df_clean.columns:
            df_clean[col] = None

    print(f"Loaded {csv_path} — mapping: {selected}")
    print(f"{len(df_clean)} rows, columns: {list(df_clean.columns)}")

    # === THÊM CỘT ĐỒNG BỘ VỚI DATABASE ===
    # Tạm thời gán tất cả bài báo thuộc nhóm 'ALL' để khớp với schema Database
    # SAU NÀY: Nếu file CSV đã có cột 'age_bucket', hãy comment dòng dưới đây lại.
    if 'age_bucket' not in df.columns:
        df['age_bucket'] = 'ALL'
        print("Đã thêm cột age_bucket (mặc định: ALL)")

    return df_clean

In [5]:
# === [3] Keep the columns that are needed for suggestions. ===
df = df.dropna(subset=["title", "content"])
df = df.reset_index(drop=True)

In [6]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 84.4 MB/s eta 0:00:00


In [ ]:
import faiss
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pickle
import os

# --- Initialize Model ---
model = SentenceTransformer("distiluse-base-multilingual-cased-v2")
D = model.get_sentence_embedding_dimension()

# --- Construction Vectors ---
texts = df["title"] + " " + df["content"]

# 1. EMITTING EMBEDDINGS (Semantic Vectors)
print("1. Calculating/Loading Semantic Embeddings...")
# Compute embeddings and store them in df["embedding"]
article_embeddings = model.encode(texts.tolist(), normalize_embeddings=True, show_progress_bar=True).astype('float32')
df["embedding"] = article_embeddings.tolist()
print("   Semantic Embeddings đã được tính và lưu vào df['embedding'].")

# 2. CONSTRUCT TF-IDF VECTORIZER
print("2. TF-IDF Vectorizer is currently under construction....")
# Sử dụng 1-gram và 2-gram cho tiếng Việt để bắt cụm từ tốt hơn
tfidf_vectorizer = TfidfVectorizer(
    lowercase=True,
    ngram_range=(1, 2),
    max_df=0.8, # Bỏ qua các từ quá phổ biến
    min_df=5 # Bỏ qua các từ quá hiếm
)
# Ma trận TF-IDF cho nội dung
tfidf_matrix = tfidf_vectorizer.fit_transform(texts)
print(f"   Ma trận TF-IDF kích thước: {tfidf_matrix.shape}")


# 3. XÂY DỰNG FAISS INDEX (Sử dụng IndexIVFFlat để tăng tốc)
def build_advanced_faiss(embeddings: np.ndarray, faiss_path: str):
    print("3. Đang xây dựng IndexIVFFlat (FAISS Nâng cấp)...")
    N_CLUSTERS = 100

    # 3.1. Tạo IndexIVFFlat
    quantizer = faiss.IndexFlatIP(D) # Index dùng để phân cụm (quantizer)
    index = faiss.IndexIVFFlat(quantizer, D, N_CLUSTERS, faiss.METRIC_INNER_PRODUCT)

    # 3.2. Huấn luyện Index (Cần một phần nhỏ dữ liệu để phân cụm)
    # Lấy mẫu ngẫu nhiên 5000 vector để huấn luyện (nếu dữ liệu nhỏ hơn, dùng toàn bộ)
    sample_size = min(len(embeddings), 5000)
    faiss.normalize_L2(embeddings) # Normalization cần cho IndexFlatIP
    index.train(embeddings[:sample_size])

    # 3.3. Thêm dữ liệu vào Index
    index.add(embeddings)
    index.nprobe = 10 # Số lượng cụm cần quét (tăng lên để chính xác hơn, giảm để nhanh hơn)

    # 3.4. Lưu Index
    faiss.write_index(index, faiss_path)
    print(f"   FAISS Index IVFFlat (ntotal={index.ntotal}) đã được lưu.")
    return index

# Define FAISS_SNAPSHOT and build the index
FAISS_SNAPSHOT = "faiss_index.bin"
index = build_advanced_faiss(article_embeddings, FAISS_SNAPSHOT)

# Tách embedding khỏi df để tiết kiệm bộ nhớ
# df_metadata đã được tạo ở các bước trước (không có cột 'embedding')

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/341 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/114 [00:00<?, ?B/s]

2_Dense/model.safetensors:   0%|          | 0.00/1.58M [00:00<?, ?B/s]

1. Calculating/Loading Semantic Embeddings...


Batches:   0%|          | 0/88 [00:00<?, ?it/s]

In [ ]:
from scipy.sparse import csr_matrix
def recommend_by_keyword_advanced(
    df_metadata: pd.DataFrame,
    model: SentenceTransformer,
    faiss_index: faiss.Index,
    tfidf_vectorizer: TfidfVectorizer,
    tfidf_matrix: csr_matrix, # Cần ma trận TF-IDF của tất cả bài báo
    query_keyword: str,
    top_n: int = 5,
    category_filter: str = None,
    top_k_faiss: int = 200 # Tăng nhẹ Top K để hỗ trợ TF-IDF
) -> pd.DataFrame:

    if not query_keyword:
        return pd.DataFrame()

    print(f"\n\u2009\u2505\u2009 Đang tìm kiếm cho: '{query_keyword}' (Lọc chuyên mục: {category_filter or 'None'})")

    # 1. TẠO QUERY VECTOR (Semantic & TF-IDF)
    query_sem_vec = model.encode(query_keyword, normalize_embeddings=True).astype('float32').reshape(1, -1)
    query_tfidf_vec = tfidf_vectorizer.transform([query_keyword]) # Vector TF-IDF cho từ khóa

    # 2. TÌM KIẾM FAISS (Lấy ứng viên tiềm năng)
    D, I = faiss_index.search(query_sem_vec, top_k_faiss)

    # Lấy metadata của Top K ứng viên
    indices = I[0].tolist()
    cands = df_metadata.iloc[indices].copy()
    cands["sim_sem"] = D[0].tolist() # Semantic Similarity

    # 3. TÍNH TF-IDF SIMILARITY (Chỉ tính trên tập Top K)
    # Lấy ma trận TF-IDF của các ứng viên
    tfidf_cands_matrix = tfidf_matrix[indices]

    # Tính cosine similarity giữa query TF-IDF và ma trận TF-IDF ứng viên
    sim_tfidf = cosine_similarity(query_tfidf_vec, tfidf_cands_matrix).ravel()
    cands["sim_tfidf"] = sim_tfidf

    # 4. LỌC CỨNG (Sửa lỗi Logic: Category)
    filtered_cands = cands.copy()
    if category_filter:
        category_filter_lower = category_filter.strip().lower()
        filtered_cands = filtered_cands[
            filtered_cands["category"].str.contains(category_filter_lower, na=False)
        ]

    if filtered_cands.empty:
        # (Giữ nguyên logic chẩn đoán đã sửa)
        top_cands_categories = cands["category"].value_counts().head(5).index.tolist()
        print(f"    (Diagnostic: 5 chuyên mục phổ biến nhất trong Top {top_k_faiss} ứng viên FAISS: {top_cands_categories})")
        print(f" Không tìm thấy kết quả nào trong chuyên mục '{category_filter}' từ Top {top_k_faiss} ứng viên FAISS.")
        return pd.DataFrame()

    # 5. RERANKING & SCORE TỔNG HỢP
    # Xử lý thời gian (Recency)
    filtered_cands["published_time_parsed"] = pd.to_datetime(
        filtered_cands["published_time"], errors="coerce"
    )
    recency_norm = filtered_cands["published_time_parsed"].rank(pct=True, ascending=False).fillna(0.0)
    filtered_cands["recency_norm"] = recency_norm

    # CÔNG THỨC SCORE MỚI (Ưu tiên Semantic > TF-IDF > Recency)
    W_SEM, W_TFIDF, W_REC = 0.5, 0.35, 0.15
    filtered_cands["score"] = (
        W_SEM * filtered_cands["sim_sem"].astype(float).fillna(0) +
        W_TFIDF * filtered_cands["sim_tfidf"].astype(float).fillna(0) +
        W_REC * filtered_cands["recency_norm"]
    )

    # 6. CHỌN TOP N
    top_df = filtered_cands.sort_values(by="score", ascending=False).head(top_n)

    # 7. Hiển thị tất cả các điểm thành phần
    return top_df[[
        "title", "url", "category", "sim_sem", "sim_tfidf", "recency_norm", "score"
    ]]


In [ ]:
# --- RUN TESTS WITH THE NEW CODE ---
print("\n--- RUN TESTS WITH THE IMPROVED MODEL ---")

# Temporary fix: Rename 'section' column to 'category' if it exists
# This is a workaround because the 'df' DataFrame does not have a 'category' column
# but it has a 'section' column which is intended to be used as category.
if 'section' in df.columns and 'category' not in df.columns:
    df_for_recommendation = df.rename(columns={'section': 'category'})
elif 'section' not in df.columns and 'category' not in df.columns:
    print("Warning: Neither 'section' nor 'category' column found in DataFrame. Category filtering will not work.")
    df_for_recommendation = df.copy()
    df_for_recommendation['category'] = None # Add a dummy column to prevent further errors if category operations are attempted.
else:
    df_for_recommendation = df.copy()

# Call the function with the new parameters.
result_ngap = recommend_by_keyword_advanced(
    df_metadata=df_for_recommendation,
    model=model,
    faiss_index=index,
    tfidf_vectorizer=tfidf_vectorizer,
    tfidf_matrix=tfidf_matrix,
    query_keyword="ngập",
    top_n=5,
    category_filter=None # Bỏ lọc chuyên mục để tìm kết quả "ngập"
)

if not result_ngap.empty:
    print("\nSuggested improvements for the keyword: 'ngập' (Top 5)")
    print(result_ngap)


In [ ]:
# === [Interactive Search]  ===

def interactive_search():
    print("="*50)
    print("INTELLIGENT NEWS SEARCH SYSTEM")
    print(" (Type 'exit' to exit the program)")
    print("="*50)

    while True:
        # 1. Nhập từ khóa
        query = input("\nEnter the keywords: ").strip()
        if query.lower() == 'exit':
            print("Thank you for your using!")
            break
        if not query:
            continue

        # 2. Giao diện Bộ lọc (Filter)
        print("\n" + "-"*30)
        filter_choice = input("Bạn có muốn lọc theo Chuyên mục không? (Nhập tên chuyên mục hoặc Enter để bỏ qua): ").strip()
        category_filter_param = filter_choice if filter_choice else None

        # 3. Tìm kiếm ứng viên (Semantic + TF-IDF Reranking)
        # Using the globally available df_for_recommendation, model, index, tfidf_vectorizer, tfidf_matrix
        res_df = recommend_by_keyword_advanced(
            df_metadata=df_for_recommendation, # This should be globally available or passed
            model=model,                       # This should be globally available or passed
            faiss_index=index,                 # This should be globally available or passed
            tfidf_vectorizer=tfidf_vectorizer, # This should be globally available or passed
            tfidf_matrix=tfidf_matrix,         # This should be globally available or passed
            query_keyword=query,
            top_n=5, # Get directly the top 5 after reranking
            category_filter=category_filter_param
        )

        if res_df.empty:
            print("Không tìm thấy bài viết nào phù hợp với từ khóa và chuyên mục đã chọn.")
            continue

        # 4. Xuất kết quả cuối cùng
        print(f"\nKẾT QUẢ CUỐI CÙNG (Top {len(res_df)}):")
        # Hiển thị định dạng scannable
        output = res_df[["title", "category", "url"]]
        for _, row in output.iterrows():
            print(f" {row['title']}")
            print(f"  ↳ Chuyên mục: {row['category']}")
            print(f"  ↳ Link: {row['url']}\n")

# Chạy chương trình tìm kiếm
interactive_search()

# Task
Define `viewed_indices` and compute `user_profile` by averaging the embeddings of the articles corresponding to `viewed_indices`, and insert this code before the cell `CyNwowUMspi6`.

## Define `viewed_indices` and `user_profile`

### Subtask:
Create a new code cell before the selected cell (`CyNwowUMspi6`) to define `viewed_indices` (e.g., with sample article indices or an empty list) and compute `user_profile` by averaging the embeddings of these `viewed_indices`.


## Summary:

### Insights or Next Steps
*   The next step involves the actual implementation of defining `viewed_indices` and computing `user_profile`.
*   The `user_profile`, once computed by averaging the embeddings of viewed articles, will be a critical input for subsequent personalized recommendation or content filtering tasks.
